In [1]:
import pandas as pd
import os
import yaml
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt 
import osmnx as ox
import folium

In [12]:
from shapely.geometry import box

In [67]:
with open("auth.yaml", 'r') as ymlfile:
    auth = yaml.safe_load(ymlfile)

In [68]:
api = SentinelAPI(auth['copernicus_open_access_hub']['username'], 
                  auth['copernicus_open_access_hub']['password'],
                  'https://scihub.copernicus.eu/dhus')

In [69]:
place_name='Girardot, Cundinamarca Colombia '

In [70]:
place = ox.gdf_from_place(place_name)

In [71]:
query_box=box(minx=place.bbox_west.values[0],
    miny=place.bbox_south.values[0],
    maxx=place.bbox_east.values[0],
    maxy=place.bbox_north.values[0])

In [100]:
m = folium.Map([place.geometry.x, place.geometry.y], zoom_start=20)

In [101]:
folium.GeoJson(place).add_to(m)

In [103]:
# m

In [88]:
place.to_json()

'{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"bbox_east": -74.6415665, "bbox_north": 4.466643, "bbox_south": 4.146643, "bbox_west": -74.9615665, "place_name": "Girardot, Cundinamarca, 252431, Colombia"}, "geometry": {"type": "Point", "coordinates": [-74.8015665, 4.306643]}}]}'

In [84]:
products = api.query(query_box.wkt,
                     date = ('20190601', '20190926'),
                     platformname = 'Sentinel-2',
                     processinglevel = 'Level-2A',
                     cloudcoverpercentage = (0,10))


In [85]:
products_gdf = api.to_geodataframe(products)

In [86]:
if not products_gdf.empty:
    products_gdf_sorted = products_gdf.sort_values(['cloudcoverpercentage'], ascending=[True])
else:
    print ('No records scences found for the filtered parameters')

In [116]:
uuid=products_gdf_sorted.index.to_list()[0]

In [120]:
title=products_gdf_sorted.loc[uuid].title

In [114]:
api.download(uuid, directory_path='./imagery')

Downloading: 100%|██████████| 1.20G/1.20G [02:57<00:00, 6.74MB/s] 
MD5 checksumming: 100%|██████████| 1.20G/1.20G [00:02<00:00, 424MB/s]


{'id': '655a2364-bbd3-4ebb-9f21-9a103afdf268',
 'title': 'S2A_MSIL2A_20190822T152641_N0213_R025_T18NVK_20190822T210428',
 'size': 1195068314,
 'md5': '625A52A0F8790078A10FB1A37510FA0B',
 'date': datetime.datetime(2019, 8, 22, 15, 26, 41, 24000),
 'footprint': 'POLYGON((-75.901794 4.523362716978575,-74.91202 4.523919106072805,-74.91211 3.53057468839868,-75.900696 3.530140808310985,-75.901794 4.523362716978575))',
 'url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('655a2364-bbd3-4ebb-9f21-9a103afdf268')/$value",
 'Online': True,
 'Creation Date': datetime.datetime(2019, 8, 23, 1, 18, 19, 849000),
 'Ingestion Date': datetime.datetime(2019, 8, 23, 1, 15, 46, 590000),
 'path': './S2A_MSIL2A_20190822T152641_N0213_R025_T18NVK_20190822T210428.zip',
 'downloaded_bytes': 1195068314}

In [123]:
import zipfile
with zipfile.ZipFile(f"imagery/{title}.zip","r") as zip_ref:
    zip_ref.extractall("imagery")